# EmpkinS IO

## Imports and Helper Functions

In [ ]:
import re
import json
from pathlib import Path

import pandas as pd
import numpy as np
import pingouin as pg

import biopsykit as bp
from biopsykit.signals.ecg import EcgProcessor

import matplotlib.pyplot as plt
import seaborn as sns

import empkins_io as eio

%load_ext autoreload
%autoreload 2
%matplotlib widget

In [ ]:
palette = bp.colors.fau_palette
sns.set_theme(context="notebook", style="ticks", palette=palette)

plt.rcParams['figure.figsize'] = (10, 5)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['mathtext.default'] = "regular"

pg.options['round'] = 3

palette

## Variable and Path Setup

In [ ]:
subject_id = "VP_01"
condition = "stress"

In [ ]:
config_file = Path("../config.json")
with open(config_file) as f:
    config_json = json.load(f)
    base_path = Path(config_json["base_path"])

base_path = Path("..").joinpath(base_path)
data_path = base_path.joinpath("data_per_subject/{}/{}".format(subject_id, condition))

ecg_path = data_path.joinpath("ecg/raw")
radar_path = data_path.joinpath("radar/raw")
video_path = data_path.joinpath("video")
timelog_path = data_path.joinpath("time_log/processed")

### Load Data

In [ ]:
ecg_files = list(sorted(ecg_path.glob("*.bin")))

In [ ]:
radar_files = list(sorted(radar_path.glob("*.mat")))

In [ ]:
index = 1

In [ ]:
ecg, fs_ecg = bp.io.nilspod.load_dataset_nilspod(ecg_files[index])

In [ ]:
radar_raw, fs_radar = eio.sensors.a04.load_data_raw(radar_files[index])
radar, fs_radar =  eio.sensors.a04.load_data(radar_files[index], datastreams=["hr", "resp"])
radar = radar["hr"]

### Plot Acc Data

In [ ]:
import matplotlib
fig, ax = plt.subplots()
ecg.filter(like="acc").plot(ax=ax)
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter("%H:%M:%S.%f"))

### Process ECG Data

In [ ]:
ep = EcgProcessor(ecg, fs_ecg)
ep.ecg_process()

hr_ecg = ep.heart_rate["Data"]

In [ ]:
bp.signals.ecg.plotting.ecg_plot(ecg_processor=ep, key="Data", figsize=(10,5))

In [ ]:
hr_radar = radar[["Heart_Rate"]]

In [ ]:
fig, ax = plt.subplots()

hr_ecg.plot(ax=ax, legend="ECG")
hr_radar.plot(ax=ax, legend="Radar")

## Just some Video/Audio Import Testing...

In [ ]:
from ipywidgets import Video, Audio

In [ ]:
video_file = video_path.joinpath("VP_06_EB_pre.mp4")

In [ ]:
Video.from_file(video_file)

In [ ]:
import librosa
from librosa.display import waveplot
import ffmpeg

In [ ]:
probe = ffmpeg.probe(video_file)

In [ ]:
video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
video_info

In [ ]:
audio = ffmpeg.input(video_file).audio

In [ ]:
out = ffmpeg.output(audio, filename=video_path.joinpath("test.mp3"))
out.run()

In [ ]:
audio, fs = librosa.load(video_path.joinpath("test.mp3"))

In [ ]:
fig, ax = plt.subplots()
waveplot(audio)